###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es la Policía Nacional de Colombia

In [28]:
import pandas as pd

###Archivo 72sg-cybi.csv

Contiene información del delito de Lesiones personales y Lesiones en accidente de Tránsito desde 01 de enero del año 2010 al 30 de abril del año 2024.

In [29]:
df = pd.read_csv("https://www.datos.gov.co/resource/72sg-cybi.csv?$limit=1500000")
df.head()

<ipython-input-29-d906eed77969>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("https://www.datos.gov.co/resource/72sg-cybi.csv?$limit=1500000")


,departamento,municipio,codigo_dane,armas_medios,fecha_hecho,genero,grupo_etario,descripci_n_conducta,cantidad
0,ANTIOQUIA,GIRARDOTA,5308000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,LESIONES PERSONALES,2
1,ANTIOQUIA,GIRARDOTA,5308000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,LESIONES PERSONALES,1
2,ANTIOQUIA,MUTATÁ,5480000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,LESIONES PERSONALES,1
3,ANTIOQUIA,NECOCLÍ,5490000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,LESIONES PERSONALES,1
4,ATLÁNTICO,BARRANQUILLA (CT),8001000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,LESIONES PERSONALES,2


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1348482 entries, 0 to 1348481
Data columns (total 9 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   departamento          1348482 non-null  object
 1   municipio             1348482 non-null  object
 2   codigo_dane           1348482 non-null  object
 3   armas_medios          1348482 non-null  object
 4   fecha_hecho           1348482 non-null  object
 5   genero                1348482 non-null  object
 6   grupo_etario          1347518 non-null  object
 7   descripci_n_conducta  1348482 non-null  object
 8   cantidad              1348482 non-null  int64 
dtypes: int64(1), object(8)
memory usage: 92.6+ MB


### Eliminación de columnas irrelevantes para el proyecto

In [31]:
relevant_cols = ['codigo_dane', 'departamento', 'municipio', 'fecha_hecho', 'genero', 'grupo_etario', 'descripci_n_conducta', 'cantidad']
personal_injury = df[relevant_cols]
personal_injury.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1348482 entries, 0 to 1348481
Data columns (total 8 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   codigo_dane           1348482 non-null  object
 1   departamento          1348482 non-null  object
 2   municipio             1348482 non-null  object
 3   fecha_hecho           1348482 non-null  object
 4   genero                1348482 non-null  object
 5   grupo_etario          1347518 non-null  object
 6   descripci_n_conducta  1348482 non-null  object
 7   cantidad              1348482 non-null  int64 
dtypes: int64(1), object(7)
memory usage: 82.3+ MB


### Convertir columna fecha_hecho a tipo date

In [32]:
# Hacer una copia explícita del DataFrame
personal_injury_ = personal_injury.copy()

# Convertir la columna 'fecha_hecho' a tipo datetime usando el formato correcto
personal_injury_['fecha_hecho'] = pd.to_datetime(personal_injury_['fecha_hecho'], format='%d/%m/%Y', errors='coerce', dayfirst=True)

# Extraer solo el año
personal_injury_.loc[:, 'fecha_hecho'] = personal_injury_['fecha_hecho'].dt.year

### Filtrar solamente conductas de lesiones personales

In [34]:
personal_injury_ = personal_injury_[personal_injury_['descripci_n_conducta'] == 'LESIONES PERSONALES']

### Verificar valores nulos

In [36]:
personal_injury_.isnull().sum()

,0
codigo_dane,0
departamento,0
municipio,0
fecha_hecho,0
genero,0
grupo_etario,960
descripci_n_conducta,0
cantidad,0


Nota: Los nulos de 'grupo_etario' los abordamos más adelante

### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [37]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['genero', 'grupo_etario', 'descripci_n_conducta', 'departamento', 'municipio']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(personal_injury_[column].unique())
    print()


Categorías en la columna 'genero':
['FEMENINO' 'MASCULINO' 'NO REPORTA' '-' 'NO REPORTADO']

Categorías en la columna 'grupo_etario':
['ADULTOS' 'ADOLESCENTES' 'MENORES' 'NO REPORTA' nan 'NO REPORTADO'
 'NO REGISTRA' '-']

Categorías en la columna 'descripci_n_conducta':
['LESIONES PERSONALES']

Categorías en la columna 'departamento':
['ANTIOQUIA' 'ATLÁNTICO' 'BOLÍVAR' 'BOYACÁ' 'CALDAS' 'CAQUETÁ' 'CAUCA'
 'CESAR' 'CHOCÓ' 'CÓRDOBA' 'CUNDINAMARCA' 'GUAJIRA' 'GUAVIARE' 'HUILA'
 'MAGDALENA' 'META' 'NARIÑO' 'PUTUMAYO' 'QUINDÍO' 'RISARALDA' 'SAN ANDRÉS'
 'SANTANDER' 'SUCRE' 'TOLIMA' 'VALLE' 'AMAZONAS' 'ARAUCA' 'CASANARE'
 'NORTE DE SANTANDER' 'VAUPÉS' 'VICHADA' 'GUAINÍA']

Categorías en la columna 'municipio':
['GIRARDOTA' 'MUTATÁ' 'NECOCLÍ' ... 'López' 'Recetor'
 'El Cantón del San Pablo']



- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [38]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [39]:
# Aplicar la función a todas las columnas categóricas
personal_injury_[categorical_col] = personal_injury_[categorical_col].astype(str)
for col in categorical_col:
    personal_injury_[col] = personal_injury_[col].apply(remove_accents_and_special_chars)

In [41]:
personal_injury_.info()

<class 'pandas.core.frame.DataFrame'>
Index: 924845 entries, 0 to 1348481
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   codigo_dane           924845 non-null  object
 1   departamento          924845 non-null  object
 2   municipio             924845 non-null  object
 3   fecha_hecho           924845 non-null  int32 
 4   genero                924845 non-null  object
 5   grupo_etario          924845 non-null  object
 6   descripci_n_conducta  924845 non-null  object
 7   cantidad              924845 non-null  int64 
dtypes: int32(1), int64(1), object(6)
memory usage: 60.0+ MB


- Mejorar consistencia de las columnas 'genero' y 'grupo_etario'

In [42]:
personal_injury_['genero'] = personal_injury_['genero'].replace({
    'NO REPORTA': 'NO REPORTADO',
    '-': 'NO REPORTADO',
    'NO REPORTADO': 'NO REPORTADO'
})


In [43]:
personal_injury_['grupo_etario'] = personal_injury_['grupo_etario'].replace({
    'NO REPORTA': 'NO REPORTADO',
    'NAN': 'NO REPORTADO',
    '-': 'NO REPORTADO',
    'NO REPORTADO': 'NO REPORTADO',
    'NO REGISTRA': 'NO REPORTADO'
})

- Codificación de algunas variables categóricas

### Ajustar columna 'codigo_dane' para que coincida con el campo 'dept_mpio_code' de la tabla municipalities de la base de datos, que guarda toda la informacion de georeferenciacion de los municipios

 -  Cargar los datos con códigos reales de los municipios

Como producto de una consulta a la base de datos del proyecto que se esta construyendo (Tablas departments y municipalities) se creo el archivo csv que se carga en la siguiente celda, y que incluye los nombres de los departamentos y municipios con sus respectivos codigos, generados por el DANE (Estos codigos son los reales)

In [47]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "codigo_dane" en el df personal_injury_

In [48]:
# Asegurarnos de que todos los valores en 'codigo_dane' sean strings
personal_injury_['codigo_dane'] = personal_injury_['codigo_dane'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = personal_injury_['codigo_dane'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

False

In [49]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [ 7  8 10]


In [50]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,no_dígitos_codigo_dane,count,percentage
0,8,792574,0.856980
1,7,132268,0.143016
2,10,3,0.000003


In [51]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = personal_injury_[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['codigo_dane']])
    print()

Muestra de registros con longitud 8:
   codigo_dane
8     13001000
9     15686000
10    15759000
11    15001000
12    17001000

Muestra de registros con longitud 7:
  codigo_dane
0     5308000
1     5308000
2     5480000
3     5490000
4     8001000

Muestra de registros con longitud 10:
       codigo_dane
450872  NO REPORTA
552854  NO REPORTA
722291  NO REPORTA



### Nota:

De lo anterior se notan claras inconsistencias en la columna 'codigo_dane' del df, el 85,7% tiene 8 dígitos (al parecer se le adicionaron 3 ceros al final del código que realmente es de 5 dígitos), el 14,3% tiene solamente 7 dígitos (al parecer el cero a la izquierda de los códigos se suprimió) y hay 3 códigos que en vez del código aparece la palabra NO REPORTA

  - Verificar si los codigo_dane de 7 dígitos corresponden a departamentos que se identifican con 1 dígito para validar la teoria de que al generar el dataset se les suprimió el cero a la izquierda

In [52]:
# Filtrar las filas donde 'codigo_dane' tiene 7 dígitos
filtrado = personal_injury_[personal_injury_['codigo_dane'].str.len() == 7]

# Obtener las categorías únicas de la columna 'departamento'
categorias_departamento = filtrado['departamento'].unique()

# Imprimir las categorías
print(categorias_departamento)

['ANTIOQUIA' 'ATLANTICO']


Efectivamente ANTIOQUIA Y ATLANTICO son los unicos departamentos que tienen codigo Dane de un dígito, 5 y 8 respectivamente.

  - Adicionar un cero a los codigo_dane de 7 dígitos

In [53]:
# Función que agrega un '0' a la izquierda si la longitud del string es 7
def add_zero_if_length_7(codigo):
    if len(codigo) == 7:
        return '0' + codigo
    return codigo

# Aplicar la función a la columna 'codigo'
personal_injury_['codigo_dane'] = personal_injury_['codigo_dane'].apply(add_zero_if_length_7)

In [54]:
# Verificar cuantos codigo_dane de 7 dígitos quedaron
len(personal_injury_[personal_injury_['codigo_dane'].str.len() == 7])

0

  - Quitar los ultimos 3 ceros de los codigo_dane que tienen 8 dígitos

In [55]:
# Función que corta los últimos 3 caracteres si la longitud del string es 8
def trim_last_3_if_length_8(codigo):
    if len(codigo) == 8:
        return codigo[:5]  # Dejar solo los primeros 5 caracteres
    return codigo

# Aplicar la función a la columna 'codigo'
personal_injury_['codigo_dane'] = personal_injury_['codigo_dane'].apply(trim_last_3_if_length_8)

In [56]:
# Verificar cuantos codigo_dane de 8 dígitos quedaron
len(personal_injury_[personal_injury_['codigo_dane'].str.len() == 8])

0

### Solucionar los codigo_dane que en el dataset fueron digitados erroneamente como NO REPORTA

In [61]:
# Mostrar los dos registros donde el codigo_dane esta errado
personal_injury_[personal_injury_['codigo_dane'].isin(['NO REPORTA'])]

,codigo_dane,departamento,municipio,fecha_hecho,genero,grupo_etario,descripci_n_conducta,cantidad
450872,NO REPORTA,GUAINIA,NO REPORTA,2016,FEMENINO,ADULTOS,LESIONES PERSONALES,1
552854,NO REPORTA,GUAINIA,NO REPORTA,2016,FEMENINO,ADULTOS,LESIONES PERSONALES,1
722291,NO REPORTA,BOYACA,NO REPORTA,2018,FEMENINO,ADULTOS,LESIONES PERSONALES,1


Nota: Dado que no existe posibilidad para conocer el municipio donde ocurrio el hecho, procedemos a eliminar el registro

In [62]:
# Borrar registros
personal_injury_ = personal_injury_[~(personal_injury_['codigo_dane'].isin(['NO REPORTADO']))]

- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

In [64]:
# Aplicar funcion para estandarizar columnas 'dept_name', 'mupio_name'
for col in dept_mpios_codes[['dept_name', 'mupio_name']]:
    dept_mpios_codes[col] = dept_mpios_codes[col].apply(remove_accents_and_special_chars)

In [65]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)
    # print(f"{label2} que no están en {label1}:")
    # print(only_in_list2)
    # print()


  - Comparar nombres de departamentos en los dataframes

In [66]:
# Comparar listas de departamento
compare_lists(personal_injury_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en personal_injury_", "Departamentos dept_mpios_codes")

Departamentos en personal_injury_ que no están en Departamentos dept_mpios_codes:
{'VALLE', 'SAN ANDRES', 'GUAJIRA'}


  - Solucionar discrepancias en nombres de departamento

In [67]:
# Diccionario de mapeo basado en los resultados de la comparación departamentos
depto_mapping = {'SAN ANDRES': 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA',
                 'VALLE': 'VALLE DEL CAUCA',
                 'GUAJIRA':'LA GUAJIRA'}

# Reemplazar los nombres incorrectos  el dataframe personal_injury_
personal_injury_['departamento'] = personal_injury_['departamento'].replace(depto_mapping)

 - Confirmar que discrepancia se solucionó

In [68]:
# Comparar listas de departamento
compare_lists(personal_injury_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en personal_injury_", "Departamentos dept_mpios_codes")

Departamentos en personal_injury_ que no están en Departamentos dept_mpios_codes:
set()


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [70]:
# Comparar listas de códigos
compare_lists(personal_injury_['codigo_dane'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en personal_injury_", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en personal_injury_ que no están en Códigos de municipios dept_mpios_codes:
{'NO REPORTA'}


  - Identifcar los registros bajo este código que no corresponde a ningun código real de municipio

In [71]:
personal_injury_[personal_injury_['codigo_dane'] == 'NO REPORTA']

,codigo_dane,departamento,municipio,fecha_hecho,genero,grupo_etario,descripci_n_conducta,cantidad
450872,NO REPORTA,GUAINIA,NO REPORTA,2016,FEMENINO,ADULTOS,LESIONES PERSONALES,1
552854,NO REPORTA,GUAINIA,NO REPORTA,2016,FEMENINO,ADULTOS,LESIONES PERSONALES,1
722291,NO REPORTA,BOYACA,NO REPORTA,2018,FEMENINO,ADULTOS,LESIONES PERSONALES,1


Son 3 municipios del departamento de GUANIA que no estan identificados, procedemos a eliminar los registros del dataset

In [72]:
# Eliminar filas donde 'codigo_dane_corr' es igual a '52000'
personal_injury_ = personal_injury_[personal_injury_['codigo_dane'] != 'NO REPORTA']

### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [75]:
# Eliminar columnas innecesarias
columns_to_drop = [ 'departamento', 'municipio', 'descripci_n_conducta']
final_personal_injury = personal_injury_.drop(columns=columns_to_drop)

In [76]:
# Adicionar columna para trazabilidad de la fuente
final_personal_injury['source_id'] = 16

In [77]:
final_personal_injury.columns

Index(['codigo_dane', 'fecha_hecho', 'genero', 'grupo_etario', 'cantidad',
       'source_id'],
      dtype='object')

In [78]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'codigo_dane': 'dane_code',
    'fecha_hecho': 'year_of_incident',
    'genero': 'sex',
    'grupo_etario': 'age_group',
    'cantidad': 'amount',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_personal_injury.rename(columns=translation_map, inplace=True)

In [79]:
#Estructura final del dataset a integrar a la base de datos
final_personal_injury.info()

<class 'pandas.core.frame.DataFrame'>
Index: 924842 entries, 0 to 1348481
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   dane_code         924842 non-null  object
 1   year_of_incident  924842 non-null  int32 
 2   sex               924842 non-null  object
 3   age_group         924842 non-null  object
 4   amount            924842 non-null  int64 
 5   source_id         924842 non-null  int64 
dtypes: int32(1), int64(2), object(3)
memory usage: 45.9+ MB


## Salvar en archivo csv en el drive

In [80]:
# Guardar en archivos CSV en el drive
final_personal_injury.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/personal_injury.csv', index=False)